<a href="https://colab.research.google.com/github/texaschikkita/Aimbot-V2/blob/main/Corrected_Version_of_Home_Sales_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
%pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done


In [3]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [4]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [5]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

# Set file to url
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True, quote="\"", escape="\"")

# Show DataFrame
df.show()


+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|               date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08 00:00:00|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13 00:00:00|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12 00:00:00|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16 00:00:00|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08 00:00:00|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa0052

In [6]:
# 2. create a temporary view of the DataFrame
df.createOrReplaceTempView("house_data")

# 3. Then, run the Spark SQL query
query = """
    SELECT date_built, ROUND(AVG(Price), 2) AS Average_Price
    FROM house_data
    WHERE Bedrooms = 4
    GROUP BY date_built
    ORDER BY date_built
"""
result = spark.sql(query)

# Show the result
result.show()


+----------+-------------+
|date_built|Average_Price|
+----------+-------------+
|      2010|    296800.75|
|      2011|     302141.9|
|      2012|    298233.42|
|      2013|    299999.39|
|      2014|    299073.89|
|      2015|    307908.86|
|      2016|    296050.24|
|      2017|    296576.69|
+----------+-------------+



In [7]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql('''
SELECT date_built, ROUND(AVG(price), 2) AS Avg_Price_3bed_3bath
FROM house_data
WHERE bedrooms = 3 AND bathrooms = 3 
GROUP BY date_built
ORDER BY date_built
''').show()

+----------+--------------------+
|date_built|Avg_Price_3bed_3bath|
+----------+--------------------+
|      2010|           292859.62|
|      2011|           291117.47|
|      2012|           293683.19|
|      2013|           295962.27|
|      2014|           290852.27|
|      2015|            288770.3|
|      2016|           290555.07|
|      2017|           292676.79|
+----------+--------------------+



In [8]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

spark.sql('''
SELECT date_built, ROUND(AVG(price), 2) AS Avg_Price_3and3_2ksqft_2story
FROM house_data
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built
''').show()

+----------+-----------------------------+
|date_built|Avg_Price_3and3_2ksqft_2story|
+----------+-----------------------------+
|      2010|                    285010.22|
|      2011|                    276553.81|
|      2012|                    307539.97|
|      2013|                    303676.79|
|      2014|                    298264.72|
|      2015|                    297609.97|
|      2016|                     293965.1|
|      2017|                    280317.58|
+----------+-----------------------------+



In [9]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql('''
SELECT ROUND(AVG(view), 2) as View_Rating
FROM house_data
WHERE Price >= 350000
''').show()

end_time =time.time()

print(f"Runtime without cache: {end_time - start_time} seconds")

+-----------+
|View_Rating|
+-----------+
|      32.26|
+-----------+

Runtime without cache: 0.5645709037780762 seconds


In [10]:
# 6(b) Or (alternate query to get view rating on each)

start_time=time.time()
query = """
    SELECT View, ROUND(AVG(Price), 2) AS Average_Price
    FROM house_data
    WHERE Price >= 350000
    GROUP BY View
    ORDER BY View
"""
end_time=time.time()

result = spark.sql(query)

# Display the result
result.show()
print(f"Runtime without cache: {end_time - start_time} seconds")

+----+-------------+
|View|Average_Price|
+----+-------------+
|   0|    403848.51|
|   1|    401044.25|
|   2|    397389.25|
|   3|     398867.6|
|   4|    399631.89|
|   5|    401471.82|
|   6|    395655.38|
|   7|    403005.77|
|   8|    398592.71|
|   9|    401393.34|
|  10|    401868.43|
|  11|    399548.12|
|  12|    401501.32|
|  13|    398917.98|
|  14|    398570.03|
|  15|     404673.3|
|  16|    399586.53|
|  17|    398474.49|
|  18|    399332.91|
|  19|    398953.17|
+----+-------------+
only showing top 20 rows

Runtime without cache: 7.843971252441406e-05 seconds


In [11]:
# 7. Cache the the temporary table house_data.
spark.sql("cache table house_data")

DataFrame[]

In [12]:
# 8. Check if the table is cached.
spark.catalog.isCached('house_data')

True

In [13]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time=time.time()
query = """
    SELECT ROUND(AVG(View), 2) AS View_Rating
    FROM house_data
    WHERE Price >= 350000
"""
end_time=time.time()

result = spark.sql(query)

# Display the result
result.show()
print(f"Runtime without cache: {end_time - start_time} seconds")

+-----------+
|View_Rating|
+-----------+
|      32.26|
+-----------+

Runtime without cache: 8.535385131835938e-05 seconds


In [15]:
# 9(b) OR (alternate query to get result for each) 
start_time = time.time()

query = """
    SELECT View, ROUND(AVG(Price), 2) AS view_rating
    FROM house_data
    GROUP BY View
    HAVING AVG(Price) >= 350000
"""
end_time=time.time()
result = spark.sql(query)

# Display the result
result.show()
print(f"Runtime with cache: {end_time - start_time} seconds")

+----+-----------+
|View|view_rating|
+----+-----------+
|  85| 1056336.74|
|  65|  736679.93|
|  53|   755214.8|
|  78| 1080649.37|
|  81| 1053472.79|
|  76| 1058802.78|
|  91| 1137372.73|
|  93| 1026006.06|
|  52|  733780.26|
|  86| 1070444.25|
|  94|  1033536.2|
|  57|   734340.5|
|  54|  798684.82|
|  96| 1017815.92|
|  92|  970402.55|
|  64|  767036.67|
|  61|  746877.59|
|  88| 1031719.35|
|  72|  780914.67|
|  59|   791453.0|
+----+-----------+
only showing top 20 rows

Runtime with cache: 7.772445678710938e-05 seconds


In [16]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").parquet("/path/to/output")


In [17]:
# 11. Read the formatted parquet data.
# Import PySpark
from pyspark.sql import SparkSession

# Initiate a SparkSession
spark = SparkSession.builder.appName("read_parquet").getOrCreate()

# Provide the path to the Parquet file or directory
parquet_file_path = "/path/to/output"

# Read the Parquet file
df = spark.read.parquet(parquet_file_path)

# Display the DataFrame
df.show()


+--------------------+-------------------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|               date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+-------------------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06 00:00:00|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09 00:00:00|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08 00:00:00|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04 00:00:00|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17 00:00:00|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a

In [18]:
# 10 (b) OR alternate method 
df.write.parquet('date_built', mode='overwrite')


# 11. (b)  OR alternate method
parquet_df=spark.read.parquet('date_built')

In [19]:
# 12. Create a temporary table for the parquet data.
df.createOrReplaceTempView('temp_table')

result = spark.sql("SELECT * FROM temp_table")

# Display the result
result.show()

+--------------------+-------------------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|               date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+-------------------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06 00:00:00|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09 00:00:00|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08 00:00:00|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04 00:00:00|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17 00:00:00|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a

UPDATED QUERY FOR #13

In [20]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()

query="""

select view, round(avg(price), 2) as average_price

from temp_table
where price >= 350000 

group by view

order by view

"""

spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|   0|    403848.51|
|   1|    401044.25|
|   2|    397389.25|
|   3|     398867.6|
|   4|    399631.89|
|   5|    401471.82|
|   6|    395655.38|
|   7|    403005.77|
|   8|    398592.71|
|   9|    401393.34|
|  10|    401868.43|
|  11|    399548.12|
|  12|    401501.32|
|  13|    398917.98|
|  14|    398570.03|
|  15|     404673.3|
|  16|    399586.53|
|  17|    398474.49|
|  18|    399332.91|
|  19|    398953.17|
+----+-------------+
only showing top 20 rows

--- 0.5715837478637695 seconds ---


PREVIOUS INCORRECT QUERY FOR #13

In [21]:
# # 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# # with the parquet DataFrame. Round your average to two decimal places. 
# # Determine the runtime and compare it to the cached version. 


# query = """
#   SELECT ROUND(AVG(view), 2) as view_rating
#   FROM temp_table
#   WHERE price >= 350000
# """

# # Timing the query without caching
# start_time = time.time()
# result = spark.sql(query)
# result.show()  # Force execution of the query now
# end_time = time.time()


# print(f"Runtime without cache: {end_time - start_time} seconds")

# # Cache the DataFrame
# df.cache()

# # Timing the query with caching
# start_time = time.time()
# result = spark.sql(query)
# result.show()  # Force execution of the query now
# end_time = time.time()

# print(f"Runtime with cache: {end_time - start_time} seconds")

In [22]:
# 14. Uncache the house_data temporary table and the df
df.unpersist()
spark.sql("uncache table house_data")

DataFrame[]

In [23]:
# 15. Check if the temporary view is cached
is_cached = spark.catalog.isCached("temp_table")

print(f"Is 'temp_table' cached? {is_cached}")


Is 'temp_table' cached? False


In [24]:
# 15 (b). Check if the house_data is no longer cached
spark.catalog.isCached("house_data")
print(f"Is 'house_data' cached? {is_cached}")

Is 'house_data' cached? False
